In [1]:
import subprocess
import os
import shutil
import os
import zipfile


def zip_folder(folder_path, zip_file_name):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the complete filepath of file in directory
                file_path = os.path.join(root, file)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, folder_path))


In [2]:
task_name = "class_data_3.2.5"
type_of_task = "function"  # "function"

solution_text = r"""
def two_sum(nums, target):
    dict = {}
    for i in range(len(nums)):
        if target - nums[i] not in dict:
            dict[nums[i]] = i
        else:
            return (dict[target - nums[i]], i)
"""


legend = r"""Задача на подумать. 

Вам дан список целых чисел и некоторое целевое число, которое представляет собой сумму двух чисел из этого списка.

Напишите функцию \verb|two_sum|, которая принимает на вход список и число и возвращает индексы тех чисел из списка, сумма которых равна целевому числу. Подразумевается, что задача имеет ровно одно решение. Дважды использовать одно и то же число нельзя. Индексы возвращаются в виде кортежа. Для решения рекомендуется использовать словарь.
"""

input_info = r"\verb|list[int]|, \verb|int|"
output_info = r"\verb|tuple[int, int]|"
additional_notes = (
    r"Ваше решение должно содержать только функцию с именем \verb|two_sum|, без её вызова или ввода/вывода данных."
)

test_cases = [
    "[2, 7, 11, 15]\n9",
    "[-2, 1, 3, 7]\n1",
    "[0, 4, 3, 0]\n0",
    "[3, 3, 4, 5]\n6",
    "[3, 3, 4, 5]\n8",
    "[1000000000, 2000000000, 3000000000, 4000000000]\n7000000000",
]


In [3]:
# Копируем папку
dirname = "new_task"
src_dir = "./sample"
dst_dir = "./" + dirname
try:
    shutil.rmtree(dst_dir)
    os.remove(f"./{task_name}.zip")
except FileNotFoundError:
    pass
shutil.copytree(src_dir, dst_dir)


'./new_task'

In [4]:
# Поменять ./new_task/solutions/main.py


with open(f"./{dirname}/solutions/main.py", mode="w", encoding="utf-8") as file:
    file.write(solution_text)


In [5]:
# Сгенерить тесты


# TODO: LLM-кой сделать тесты
def create_test_cases() -> list[str]:
    pass


for i in range(len(test_cases)):
    with open(f"./{dirname}/tests/{i + 1:02}", mode="w", encoding="utf-8") as file_test:
        file_test.write(test_cases[i])


In [6]:
# Сгенерить ответы


# К этому моменту в папке ответов еще нет, так что это количество именно тестов
n_tests = len([x for x in os.listdir(f"./{dirname}/tests/") if not x.endswith(".a")])


if type_of_task == "function":
    with open(f"./{dirname}/files/participantSolution.py", mode="w", encoding="utf-8") as file:
        file.write(solution_text)

    commands = [
        f"python ./{dirname}/files/run_tests.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]

    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)

    subprocess.run(f"del ./{dirname}/files/participantSolution.py", shell=True, text=True, capture_output=True)
else:
    # basic prog task
    commands = [
        f"python ./{dirname}/solutions/main.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]
    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)
        print(process)


In [7]:
# Копирование примера
import shutil

src_test = f"./{dirname}/tests/01"
src_ans = f"./{dirname}/tests/01.a"
dst_test = f"./{dirname}/statements/russian/example.01"
dst_ans = f"./{dirname}/statements/russian/example.01.a"
shutil.copy(src_test, dst_test)
shutil.copy(src_ans, dst_ans)


'./new_task/statements/russian/example.01.a'

In [8]:
# Изменяем problem.xml
with open(f"./{dirname}/problem.xml", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_task_name", task_name)
content = content.replace("placeholder_ntests", str(n_tests))

task_price = 1
if task_price > 1:
    content = content.replace("checker_1.py", f"checker_{task_price}.py")
    content = content.replace("files/checker_1.py", f"checker_{task_price}.py")

with open(f"./{dirname}/problem.xml", mode="w", encoding="utf-8") as file:
    file.write(content)


In [9]:
# Поменять условие
with open(f"./{dirname}/statements/russian/problem.tex", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_legend", legend)
content = content.replace("placeholder_input", input_info)
content = content.replace("placeholder_output", output_info)
content = content.replace("placeholder_note", additional_notes)

with open(f"./{dirname}/statements/russian/problem.tex", mode="w", encoding="utf-8") as file:
    file.write(content)


In [10]:
zip_folder(f"{dst_dir}", f"{task_name}.zip")
